In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from sklearn.model_selection import train_test_split

In [ ]:
import os
import numpy as np
from PIL import Image

# -----------------------------
# Directories
# -----------------------------
image_dir = "/kaggle/input/global-land-cover-mapping-openearthmap/images/train"
label_dir = "/kaggle/input/global-land-cover-mapping-openearthmap/label/train"
TARGET_SIZE = (128, 128)

# -----------------------------
# Class Mapping: 9 → 4
# Example mapping (adjust to your classes):
# 0 → 0 (background)
# 1,2 → 1
# 3,4 → 2
# 5,6,7,8 → 3
# -----------------------------
def map_9_to_4(lbl):
    mapped = np.zeros_like(lbl)
    mapped[np.isin(lbl, [1, 2])] = 1
    mapped[np.isin(lbl, [3, 4])] = 2
    mapped[np.isin(lbl, [5, 6, 7, 8])] = 3
    return mapped

# -----------------------------
# Load Dataset
# -----------------------------
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(".tif")])
label_files = sorted([f for f in os.listdir(label_dir) if f.endswith(".tif")])

X, Y = [], []

for img_file, lbl_file in zip(image_files, label_files):
    img_path = os.path.join(image_dir, img_file)
    lbl_path = os.path.join(label_dir, lbl_file)
    
    # Load images
    img = Image.open(img_path).convert("RGB")
    lbl = Image.open(lbl_path)
    
    # Resize
    img = img.resize(TARGET_SIZE, Image.BILINEAR)
    lbl = lbl.resize(TARGET_SIZE, Image.NEAREST)
    
    # Convert to arrays
    img_arr = np.array(img, dtype=np.float32) / 255.0
    lbl_arr = np.array(lbl, dtype=np.int32)
    
    # Map 9 classes → 4 classes
    lbl_arr = map_9_to_4(lbl_arr)
    
    X.append(img_arr)
    Y.append(lbl_arr)

X = np.array(X, dtype=np.float32)  # (N, H, W, 3)
Y = np.array(Y, dtype=np.int32)    # (N, H, W)

print("Images shape:", X.shape)
print("Labels shape:", Y.shape)


In [ ]:
image_dir_val = "/kaggle/input/global-land-cover-mapping-openearthmap/images/val"
label_dir_val = "/kaggle/input/global-land-cover-mapping-openearthmap/label/val"


TARGET_SIZE = (128, 128)

# Get sorted lists so images and labels align
image_files_val = sorted([f for f in os.listdir(image_dir_val) if f.endswith(".tif")])
label_files_val = sorted([f for f in os.listdir(label_dir_val) if f.endswith(".tif")])

X_val = []
Y_val = []

for img_file, lbl_file in zip(image_files_val, label_files_val):
    img_path = os.path.join(image_dir_val, img_file)
    lbl_path = os.path.join(label_dir_val, lbl_file)

    
    img = Image.open(img_path).convert("RGB")  # Ensure 3 channels
    lbl = Image.open(lbl_path)                 # Mask can stay single channel

   
    img = img.resize(TARGET_SIZE, Image.BILINEAR)
    lbl = lbl.resize(TARGET_SIZE, Image.NEAREST)

   
    img_arr = np.array(img, dtype=np.float32) / 255.0  # normalize [0,1]
    lbl_arr = np.array(lbl, dtype=np.int32) 
    
    lbl_arr = map_9_to_4(lbl_arr)  

    X_val.append(img_arr)
    Y_val.append(lbl_arr)

X_val = np.array(X_val, dtype=np.float32)   # (N, H, W, 3)
Y_val = np.array(Y_val, dtype=np.int32)    

print("Image dataset shape:", X_val.shape)
print("Label dataset shape:", Y_val.shape)


In [ ]:
num_classes = len(np.unique(Y)) 
print("Num classes:", num_classes)

In [ ]:
def visualize_sample(idx):
    plt.figure(figsize=(10,5))

    # Input image
    plt.subplot(1,2,1)
    plt.title("Input Image")
    plt.imshow(X[idx])

    # Label mask
    plt.subplot(1,2,2)
    plt.title("Mask")
    plt.imshow(Y[idx].squeeze(),cmap='tab20')  # 'jet' shows classes with colors
    plt.colorbar()
    plt.show()

# Example: visualize the 3rd image–mask pair
visualize_sample(3)



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet(input_shape=(128,128,3), num_classes=4):
    inputs = layers.Input(shape=input_shape)
    
    # --- Encoder ---
    conv1 = layers.Conv2D(64, 3, padding='same', activation='relu')(inputs)
    conv1 = layers.Conv2D(64, 3, padding='same', activation='relu')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2,2))(conv1)
    
    conv2 = layers.Conv2D(128, 3, padding='same', activation='relu')(pool1)
    conv2 = layers.Conv2D(128, 3, padding='same', activation='relu')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2,2))(conv2)
    
    conv3 = layers.Conv2D(256, 3, padding='same', activation='relu')(pool2)
    conv3 = layers.Conv2D(256, 3, padding='same', activation='relu')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2,2))(conv3)
    
    # --- Bridge ---
    conv4 = layers.Conv2D(512, 3, padding='same', activation='relu')(pool3)
    conv4 = layers.Conv2D(512, 3, padding='same', activation='relu')(conv4)
    drop4 = layers.Dropout(0.5)(conv4)
    
    # --- Decoder ---
    up5 = layers.Conv2DTranspose(256, 2, strides=(2,2), padding='same')(drop4)
    merge5 = layers.concatenate([conv3, up5], axis=3)
    conv5 = layers.Conv2D(256, 3, padding='same', activation='relu')(merge5)
    conv5 = layers.Conv2D(256, 3, padding='same', activation='relu')(conv5)
    
    up6 = layers.Conv2DTranspose(128, 2, strides=(2,2), padding='same')(conv5)
    merge6 = layers.concatenate([conv2, up6], axis=3)
    conv6 = layers.Conv2D(128, 3, padding='same', activation='relu')(merge6)
    conv6 = layers.Conv2D(128, 3, padding='same', activation='relu')(conv6)
    
    up7 = layers.Conv2DTranspose(64, 2, strides=(2,2), padding='same')(conv6)
    merge7 = layers.concatenate([conv1, up7], axis=3)
    conv7 = layers.Conv2D(64, 3, padding='same', activation='relu')(merge7)
    conv7 = layers.Conv2D(64, 3, padding='same', activation='relu')(conv7)
    
    # --- Output ---
    outputs = layers.Conv2D(num_classes, 1, activation='softmax')(conv7)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Example usage
model = unet()
model.summary()


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np

num_classes = 4  # Your 4 classes

# -----------------------------
# Custom Mean IoU Metric for Sparse Labels
# -----------------------------
class SparseMeanIoU(tf.keras.metrics.Metric):
    def __init__(self, num_classes=num_classes, name="mean_iou", **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.mean_iou_metric = tf.keras.metrics.MeanIoU(num_classes=num_classes)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_classes = tf.argmax(y_pred, axis=-1)  # Convert predictions to class indices
        self.mean_iou_metric.update_state(y_true, y_pred_classes, sample_weight)

    def result(self):
        return self.mean_iou_metric.result()

    def reset_state(self):
        self.mean_iou_metric.reset_state()

# -----------------------------
# Combined Dice + Focal Loss
# -----------------------------
class_weights = np.ones(num_classes)  # Can adjust if some classes are rare

def multiclass_dice(y_true, y_pred, num_classes=num_classes, smooth=1e-6):
    y_true = tf.cast(y_true, tf.int32)
    y_true_onehot = tf.one_hot(y_true, depth=num_classes, dtype=tf.float32)
    y_pred_prob = tf.nn.softmax(y_pred, axis=-1)
    y_true_f = tf.reshape(y_true_onehot, [-1, num_classes])
    y_pred_f = tf.reshape(y_pred_prob, [-1, num_classes])
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    union = tf.reduce_sum(y_true_f + y_pred_f, axis=0)
    dice_per_class = (2. * intersection + smooth) / (union + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_loss(y_true, y_pred):
    return 1 - multiclass_dice(y_true, y_pred)

def focal_loss(y_true, y_pred, alpha=0.25, gamma=2.0):
    y_true = tf.cast(y_true, tf.int32)
    y_true_onehot = tf.one_hot(y_true, depth=num_classes, dtype=tf.float32)
    y_pred_prob = tf.nn.softmax(y_pred, axis=-1)
    y_pred_prob = tf.clip_by_value(y_pred_prob, 1e-8, 1-1e-8)
    
    ce_loss = -y_true_onehot * tf.math.log(y_pred_prob)
    pt = tf.where(tf.equal(y_true_onehot, 1), y_pred_prob, 1 - y_pred_prob)
    focal_weight = alpha * tf.pow(1 - pt, gamma) * class_weights
    focal_loss_val = focal_weight * ce_loss
    return tf.reduce_mean(tf.reduce_sum(focal_loss_val, axis=-1))

def combined_loss(y_true, y_pred):
    return 0.5 * dice_loss(y_true, y_pred) + 0.5 * focal_loss(y_true, y_pred)

# -----------------------------
# Sparse Categorical Accuracy
# -----------------------------
def sparse_categorical_accuracy_custom(y_true, y_pred):
    y_pred_classes = tf.argmax(y_pred, axis=-1)
    return tf.reduce_mean(tf.cast(tf.equal(tf.cast(y_true, tf.int64), y_pred_classes), tf.float32))



# -----------------------------
# Compile Model
# -----------------------------
model.compile(
    optimizer=Adam(1e-4),
    loss=combined_loss,
    metrics=[sparse_categorical_accuracy_custom, SparseMeanIoU(num_classes=num_classes)]
)

# -----------------------------
# Callbacks
# -----------------------------
callbacks = [
    ModelCheckpoint(
        "sat_map_seg_best.h5",
        monitor="val_mean_iou",
        mode="max",
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        patience=12,
        monitor="val_mean_iou",
        mode="max",
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        factor=0.5,
        patience=4,
        monitor="val_mean_iou",
        mode="max",
        min_lr=1e-6,
        verbose=1,
        cooldown=1
    )
]

# -----------------------------
# Train the Model
# -----------------------------
history = model.fit(
    X, Y,  # X, Y are your training images and labels
    validation_data=(X_val, Y_val),
    epochs=100,
    batch_size=8,
    callbacks=callbacks,
    verbose=1,
    shuffle=True
)


In [ ]:
idx = 99
test_img = X_val[idx]  # shape: (H, W, 3)
test_mask = Y_val[idx] # shape: (H, W)

# Add batch dimension
input_img = np.expand_dims(test_img, axis=0)  # shape: (1, H, W, 3)

# Predict mask probabilities for all classes
pred_mask_prob_full = model.predict(input_img)[0]  # shape: (H, W, 9)

print("Predicted mask shape:", pred_mask_prob_full.shape)
print("Min prob:", pred_mask_prob_full.min())
print("Max prob:", pred_mask_prob_full.max())

# Convert probabilities to predicted class
pred_mask = np.argmax(pred_mask_prob_full, axis=-1)  # shape: (H, W), values 0..8

# Plot input image, true mask, predicted mask
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(test_img)
plt.title("Input Image")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(test_mask, cmap='tab20')
plt.title("Ground Truth Mask")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(pred_mask, cmap='tab20')
plt.title("Predicted Mask")
plt.axis('off')

plt.tight_layout()
plt.show()
